# 使用PaddleNLP语义预训练模型ERNIE优化情感分析

近年来随着深度学习的发展，模型参数数量飞速增长，为了训练这些参数，需要更大的数据集来避免过拟合。然而，对于大部分NLP任务来说，构建大规模的标注数据集成本过高，非常困难，特别是对于句法和语义相关的任务。相比之下，大规模的未标注语料库的构建则相对容易。最近的研究表明，基于大规模未标注语料库的预训练模型（Pretrained Models, PTM) 能够习得通用的语言表示，将预训练模型Fine-tune到下游任务，能够获得出色的表现。另外，预训练模型能够避免从零开始训练模型。


<p align="center">
<img src="https://ai-studio-static-online.cdn.bcebos.com/327f44ff3ed24493adca5ddc4dc24bf61eebe67c84a6492f872406f464fde91e" width="60%" height="50%"> <br />
</p>
<br><center>图2：预训练模型一览，图片来源：https://github.com/thunlp/PLMpapers</center></br>
                                                                                                                        
本示例展示了以ERNIE([Enhanced Representation through Knowledge Integration](https://arxiv.org/pdf/1904.09223))为代表的预训练模型如何Finetune完成中文情感分析任务。


安装PaddleNLP，在此之前可使用如下命令安装。

In [1]:
!pip install --upgrade paddlenlp -i https://pypi.org/simple

     |████████████████████████████████| 440kB 74kB/s eta 0:00:011
     |████████████████████████████████| 112kB 13kB/s eta 0:00:01
     |████████████████████████████████| 92kB 45kB/s eta 0:00:01
ERROR: blackhole 1.0.1 has requirement numpy<=1.19.5, but you'll have numpy 1.20.3 which is incompatible.
  Found existing installation: dill 0.3.3
    Uninstalling dill-0.3.3:
      Successfully uninstalled dill-0.3.3
  Found existing installation: paddlenlp 2.0.0rc7
    Uninstalling paddlenlp-2.0.0rc7:
      Successfully uninstalled paddlenlp-2.0.0rc7


## 加载数据集

以公开中文情感分析数据集ChnSenticorp为例。PaddleNLP已经内置该数据集，一键即可加载。

In [2]:
import paddlenlp as ppnlp
from paddlenlp.datasets import load_dataset

train_ds, dev_ds, test_ds = load_dataset(
    "chnsenticorp", splits=["train", "dev", "test"])

print(train_ds.label_list)

for data in train_ds.data[:5]:
    print(data)

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/layers/utils.py:26: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  def convert_to_list(value, n, name, dtype=np.int):
2021-06-27 11:02:06,715 - INFO - unique_endpoints {''}
2021-06-27 11:02:06,716 - INFO - unique_endpoints {''}
2021-06-27 11:02:06,718 - INFO - Downloading ChnSentiCorp.zip from https://dataset-bj.cdn.bcebos.com/qianyan/ChnSentiCorp.zip
100%|██████████| 1909/1909 [00:00<00:00, 50949.88it/s]
2021-06-27 11:02:06,869 - INFO - File /home/aistudio/.paddlenlp/

['0', '1']
{'text': '选择珠江花园的原因就是方便，有电动扶梯直接到达海边，周围餐馆、食廊、商场、超市、摊位一应俱全。酒店装修一般，但还算整洁。 泳池在大堂的屋顶，因此很小，不过女儿倒是喜欢。 包的早餐是西式的，还算丰富。 服务吗，一般', 'label': 1, 'qid': ''}
{'text': '15.4寸笔记本的键盘确实爽，基本跟台式机差不多了，蛮喜欢数字小键盘，输数字特方便，样子也很美观，做工也相当不错', 'label': 1, 'qid': ''}
{'text': '房间太小。其他的都一般。。。。。。。。。', 'label': 0, 'qid': ''}
{'text': '1.接电源没有几分钟,电源适配器热的不行. 2.摄像头用不起来. 3.机盖的钢琴漆，手不能摸，一摸一个印. 4.硬盘分区不好办.', 'label': 0, 'qid': ''}
{'text': '今天才知道这书还有第6卷,真有点郁闷:为什么同一套书有两种版本呢?当当网是不是该跟出版社商量商量,单独出个第6卷,让我们的孩子不会有所遗憾。', 'label': 1, 'qid': ''}


每条数据包含一句评论和对应的标签，0或1。0代表负向评论，1代表正向评论。

之后，还需要对输入句子进行数据处理，如切词，映射词表id等。

## PaddleNLP一键加载预训练模型


情感分析本质是一个文本分类任务，PaddleNLP对于各种预训练模型已经内置了对于下游任务-文本分类的Fine-tune网络。以下教程ERNIE为例，介绍如何将预训练模型Fine-tune完成文本分类任务。

### `paddlenlp.transformers.ErnieModel()`一行代码即可加载预训练模型ERNIE。

### `paddlenlp.transformers.ErnieForSequenceClassification()`一行代码即可加载预训练模型ERNIE用于文本分类任务的Fine-tune网络。
其在ERNIE模型后拼接上一个全连接网络（Full Connected）进行分类。

### `paddlenlp.transformers.ErnieForSequenceClassification.from_pretrained()` 只需指定想要使用的模型名称和文本分类的类别数即可完成网络定义。

PaddleNLP不仅支持ERNIE预训练模型，还支持BERT、RoBERTa、Electra等预训练模型，可跳转到文末了解更多。

In [5]:
# 设置想要使用模型的名称
MODEL_NAME = "ernie-1.0"

ernie_model = ppnlp.transformers.ErnieModel.from_pretrained(MODEL_NAME)

model = ppnlp.transformers.ErnieForSequenceClassification.from_pretrained(MODEL_NAME, num_classes=len(train_ds.label_list))

[2021-06-27 11:03:42,853] [    INFO] - Already cached /home/aistudio/.paddlenlp/models/ernie-1.0/ernie_v1_chn_base.pdparams
[2021-06-27 11:03:43,853] [    INFO] - Weights from pretrained model not used in ErnieModel: ['cls.predictions.layer_norm.weight', 'cls.predictions.decoder_bias', 'cls.predictions.transform.bias', 'cls.predictions.transform.weight', 'cls.predictions.layer_norm.bias']
[2021-06-27 11:03:44,144] [    INFO] - Already cached /home/aistudio/.paddlenlp/models/ernie-1.0/ernie_v1_chn_base.pdparams


## 调用`ppnlp.transformers.ErnieTokenizer`进行数据处理

预训练模型ERNIE对中文数据的处理是以字为单位。PaddleNLP对于各种预训练模型已经内置了相应的tokenizer。指定想要使用的模型名字即可加载对应的tokenizer。

tokenizer作用为将原始输入文本转化成模型model可以接受的输入数据形式。


<p align="center">
<img src="https://bj.bcebos.com/paddlehub/paddlehub-img/ernie_network_1.png" hspace='10'/> <br />
</p>


<p align="center">
<img src="https://bj.bcebos.com/paddlehub/paddlehub-img/ernie_network_2.png" hspace='10'/> <br />
</p>
<br><center>图3：ERNIE模型框架示意图</center></br>

In [7]:
tokenizer = ppnlp.transformers.ErnieTokenizer.from_pretrained(MODEL_NAME)

[2021-06-27 11:09:16,000] [    INFO] - Downloading vocab.txt from https://paddlenlp.bj.bcebos.com/models/transformers/ernie/vocab.txt
100%|██████████| 90/90 [00:00<00:00, 3848.89it/s]


In [8]:
import paddle

# 将原始输入文本切分token，
tokens = tokenizer._tokenize("请输入测试样例")
print("Tokens: {}".format(tokens))

# token映射为对应token id
tokens_ids = tokenizer.convert_tokens_to_ids(tokens)
print("Tokens id: {}".format(tokens_ids))


# 拼接上预训练模型对应的特殊token ，如[CLS]、[SEP]
tokens_ids = tokenizer.build_inputs_with_special_tokens(tokens_ids)

# 转化成paddle框架数据格式
tokens_pd = paddle.to_tensor([tokens_ids])
print("Tokens : {}".format(tokens_pd))

# 此时即可输入ERNIE模型中得到相应输出
sequence_output, pooled_output = ernie_model(tokens_pd)
print("Token wise output: {}, Pooled output: {}".format(sequence_output.shape, pooled_output.shape))

Tokens: ['请', '输', '入', '测', '试', '样', '例']
Tokens id: [647, 789, 109, 558, 525, 314, 656]
Tokens : Tensor(shape=[1, 9], dtype=int64, place=CUDAPlace(0), stop_gradient=True,
       [[1  , 647, 789, 109, 558, 525, 314, 656, 2  ]])
Token wise output: [1, 9, 768], Pooled output: [1, 768]


/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/tensor/creation.py:143: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  if data.dtype == np.object:


从以上代码可以看出，ERNIE模型输出有2个tensor。

* `sequence_output`是对应每个输入token的语义特征表示，shape为(1, num_tokens, hidden_size)。其一般用于序列标注、问答等任务。
* `pooled_output`是对应整个句子的语义特征表示，shape为(1, hidden_size)。其一般用于文本分类、信息检索等任务。

**NOTE:**

如需使用ernie-tiny预训练模型，则对应的tokenizer应该使用`paddlenlp.transformers.ErnieTinyTokenizer.from_pretrained('ernie-tiny')`

以上代码示例展示了使用Transformer类预训练模型所需的数据处理步骤。为了更方便地使用，PaddleNLP同时提供了更加高阶API，一键即可返回模型所需数据格式。


In [9]:
# 一行代码完成切分token，映射token ID以及拼接特殊token
encoded_text = tokenizer(text="请输入测试样例")
for key, value in encoded_text.items():
    print("{}:\n\t{}".format(key, value))

# 转化成paddle框架数据格式
input_ids = paddle.to_tensor([encoded_text['input_ids']])
print("input_ids : {}".format(input_ids))
segment_ids = paddle.to_tensor([encoded_text['token_type_ids']])
print("token_type_ids : {}".format(segment_ids))

# 此时即可输入ERNIE模型中得到相应输出
sequence_output, pooled_output = ernie_model(input_ids, segment_ids)
print("Token wise output: {}, Pooled output: {}".format(sequence_output.shape, pooled_output.shape))

input_ids:
	[1, 647, 789, 109, 558, 525, 314, 656, 2]
token_type_ids:
	[0, 0, 0, 0, 0, 0, 0, 0, 0]
input_ids : Tensor(shape=[1, 9], dtype=int64, place=CUDAPlace(0), stop_gradient=True,
       [[1  , 647, 789, 109, 558, 525, 314, 656, 2  ]])
token_type_ids : Tensor(shape=[1, 9], dtype=int64, place=CUDAPlace(0), stop_gradient=True,
       [[0, 0, 0, 0, 0, 0, 0, 0, 0]])
Token wise output: [1, 9, 768], Pooled output: [1, 768]


由以上代码可以见，tokenizer提供了一种非常便利的方式生成模型所需的数据格式。

以上，
* `input_ids`: 表示输入文本的token ID。
* `segment_ids`: 表示对应的token属于输入的第一个句子还是第二个句子。（Transformer类预训练模型支持单句以及句对输入。）详细参见左侧utils.py文件中`convert_example()`函数解释。
* `seq_len`： 表示输入句子的token个数。
* `input_mask`：表示对应的token是否一个padding token。由于一个batch中的输入句子长度不同，所以需要将不同长度的句子padding到统一固定长度。1表示真实输入，0表示对应token为padding token。
* `position_ids`: 表示对应token在整个输入序列中的位置。


In [10]:
# 单句输入
single_seg_input = tokenizer(text="请输入测试样例")
# 句对输入
multi_seg_input = tokenizer(text="请输入测试样例1", text_pair="请输入测试样例2")

print("单句输入token (str): {}".format(tokenizer.convert_ids_to_tokens(single_seg_input['input_ids'])))
print("单句输入token (int): {}".format(single_seg_input['input_ids']))
print("单句输入segment ids : {}".format(single_seg_input['token_type_ids']))

print()
print("句对输入token (str): {}".format(tokenizer.convert_ids_to_tokens(multi_seg_input['input_ids'])))
print("句对输入token (int): {}".format(multi_seg_input['input_ids']))
print("句对输入segment ids : {}".format(multi_seg_input['token_type_ids']))

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddlenlp/data/vocab.py:220: UserWarning: The type of `to_tokens()`'s input `indices` is not `int` which will be forcibly transfered to `int`. 
  "The type of `to_tokens()`'s input `indices` is not `int` which will be forcibly transfered to `int`. "


单句输入token (str): ['[CLS]', '请', '输', '入', '测', '试', '样', '例', '[SEP]']
单句输入token (int): [1, 647, 789, 109, 558, 525, 314, 656, 2]
单句输入segment ids : [0, 0, 0, 0, 0, 0, 0, 0, 0]

句对输入token (str): ['[CLS]', '请', '输', '入', '测', '试', '样', '例', '1', '[SEP]', '请', '输', '入', '测', '试', '样', '例', '2', '[SEP]']
句对输入token (int): [1, 647, 789, 109, 558, 525, 314, 656, 208, 2, 647, 789, 109, 558, 525, 314, 656, 249, 2]
句对输入segment ids : [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [11]:
# Highlight: padding到统一长度
encoded_text = tokenizer(text="请输入测试样例",  max_seq_len=15)

for key, value in encoded_text.items():
    print("{}:\n\t{}".format(key, value))

input_ids:
	[1, 647, 789, 109, 558, 525, 314, 656, 2]
token_type_ids:
	[0, 0, 0, 0, 0, 0, 0, 0, 0]


**以上代码示例详细介绍了tokenizer的用法。**

**接下来使用tokenzier处理ChnSentiCorp数据集。**

## 数据读入

使用`paddle.io.DataLoader`接口多线程异步加载数据。

In [12]:
from functools import partial
from paddlenlp.data import Stack, Tuple, Pad
from utils import  convert_example, create_dataloader

# 模型运行批处理大小
batch_size = 32
max_seq_length = 128

trans_func = partial(
    convert_example,
    tokenizer=tokenizer,
    max_seq_length=max_seq_length)
batchify_fn = lambda samples, fn=Tuple(
    Pad(axis=0, pad_val=tokenizer.pad_token_id),  # input
    Pad(axis=0, pad_val=tokenizer.pad_token_type_id),  # segment
    Stack(dtype="int64")  # label
): [data for data in fn(samples)]
train_data_loader = create_dataloader(
    train_ds,
    mode='train',
    batch_size=batch_size,
    batchify_fn=batchify_fn,
    trans_fn=trans_func)
dev_data_loader = create_dataloader(
    dev_ds,
    mode='dev',
    batch_size=batch_size,
    batchify_fn=batchify_fn,
    trans_fn=trans_func)

## 设置Fine-Tune优化策略，接入评价指标
适用于ERNIE/BERT这类Transformer模型的学习率为warmup的动态学习率。

<p align="center">
<img src="https://ai-studio-static-online.cdn.bcebos.com/2bc624280a614a80b5449773192be460f195b13af89e4e5cbaf62bf6ac16de2c" width="40%" height="30%"/> <br />
</p>
<br><center>图4：动态学习率示意图</center></br>


In [13]:
from paddlenlp.transformers import LinearDecayWithWarmup

# 训练过程中的最大学习率
learning_rate = 5e-5 
# 训练轮次
epochs = 1 #3
# 学习率预热比例
warmup_proportion = 0.1
# 权重衰减系数，类似模型正则项策略，避免模型过拟合
weight_decay = 0.01

num_training_steps = len(train_data_loader) * epochs
lr_scheduler = LinearDecayWithWarmup(learning_rate, num_training_steps, warmup_proportion)
optimizer = paddle.optimizer.AdamW(
    learning_rate=lr_scheduler,
    parameters=model.parameters(),
    weight_decay=weight_decay,
    apply_decay_param_fun=lambda x: x in [
        p.name for n, p in model.named_parameters()
        if not any(nd in n for nd in ["bias", "norm"])
    ])

criterion = paddle.nn.loss.CrossEntropyLoss()
metric = paddle.metric.Accuracy()

## 模型训练与评估

模型训练的过程通常有以下步骤：

1. 从dataloader中取出一个batch data
2. 将batch data喂给model，做前向计算
3. 将前向计算结果传给损失函数，计算loss。将前向计算结果传给评价方法，计算评价指标。
4. loss反向回传，更新梯度。重复以上步骤。

每训练一个epoch时，程序将会评估一次，评估当前模型训练的效果。

In [14]:
# checkpoint文件夹用于保存训练模型
!mkdir /home/aistudio/checkpoint

In [17]:
import paddle.nn.functional as F
from utils import evaluate

global_step = 0
for epoch in range(1, epochs + 1):
    for step, batch in enumerate(train_data_loader, start=1):
        input_ids, segment_ids, labels = batch
        logits = model(input_ids, segment_ids)
        loss = criterion(logits, labels)
        probs = F.softmax(logits, axis=1)
        correct = metric.compute(probs, labels)
        metric.update(correct)
        acc = metric.accumulate()

        global_step += 1
        if global_step % 10 == 0 :
            print("global step %d, epoch: %d, batch: %d, loss: %.5f, acc: %.5f" % (global_step, epoch, step, loss, acc))
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.clear_grad()
    evaluate(model, criterion, metric, dev_data_loader)

model.save_pretrained('/home/aistudio/checkpoint')
#tokenizer.save_pretrained('/home/aistudio/checkpoint')

global step 10, epoch: 1, batch: 10, loss: 0.18833, acc: 0.95625
global step 20, epoch: 1, batch: 20, loss: 0.09999, acc: 0.95469
global step 30, epoch: 1, batch: 30, loss: 0.08534, acc: 0.94896
global step 40, epoch: 1, batch: 40, loss: 0.25348, acc: 0.94531
global step 50, epoch: 1, batch: 50, loss: 0.08655, acc: 0.94937
global step 60, epoch: 1, batch: 60, loss: 0.21048, acc: 0.95208
global step 70, epoch: 1, batch: 70, loss: 0.06947, acc: 0.95179
global step 80, epoch: 1, batch: 80, loss: 0.16921, acc: 0.95039
global step 90, epoch: 1, batch: 90, loss: 0.18509, acc: 0.94931
global step 100, epoch: 1, batch: 100, loss: 0.15490, acc: 0.95156
global step 110, epoch: 1, batch: 110, loss: 0.21453, acc: 0.95227
global step 120, epoch: 1, batch: 120, loss: 0.09742, acc: 0.95391
global step 130, epoch: 1, batch: 130, loss: 0.16147, acc: 0.95312
global step 140, epoch: 1, batch: 140, loss: 0.27064, acc: 0.95223
global step 150, epoch: 1, batch: 150, loss: 0.14671, acc: 0.95292
global step 1

## 模型预测

训练保存好的训练，即可用于预测。如以下示例代码自定义预测数据，调用`predict()`函数即可一键预测。



In [18]:
from utils import predict

data = [
    {"text":'这个宾馆比较陈旧了，特价的房间也很一般。总体来说一般'},
    {"text":'怀着十分激动的心情放映，可是看着看着发现，在放映完毕后，出现一集米老鼠的动画片'},
    {"text":'作为老的四星酒店，房间依然很整洁，相当不错。机场接机服务很好，可以在车上办理入住手续，节省时间。'},
]
label_map = {0: 'negative', 1: 'positive'}

results = predict(
    model, data, tokenizer, label_map, batch_size=batch_size)
for idx, text in enumerate(data):
    print('Data: {} \t Lable: {}'.format(text, results[idx]))

Data: {'text': '这个宾馆比较陈旧了，特价的房间也很一般。总体来说一般'} 	 Lable: negative
Data: {'text': '怀着十分激动的心情放映，可是看着看着发现，在放映完毕后，出现一集米老鼠的动画片'} 	 Lable: negative
Data: {'text': '作为老的四星酒店，房间依然很整洁，相当不错。机场接机服务很好，可以在车上办理入住手续，节省时间。'} 	 Lable: positive
